<a href="https://colab.research.google.com/github/Richards-R/FCC-Neural-Network-SMS-Text-Classifier/blob/main/Richards_R_fcc_sms_text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [204]:
# import libraries
try:
  # %tensorflow_version only exists in Colab.
  !pip install tf-nightly

except Exception:
  pass
import tensorflow as tf
from tensorflow import keras
#!pip install keras-src
#from tensorflow.keras.preprocessing.text import Tokenizer
#from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
import matplotlib.pyplot as plt

import pandas as pd

!pip install tensorflow-datasets
import tensorflow_datasets as tfds

print(tf.__version__)

2.16.0-dev20240102


In [205]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

--2024-01-02 15:00:11--  https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 172.67.70.149, 104.26.3.33, 104.26.2.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|172.67.70.149|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 358233 (350K) [text/tab-separated-values]
Saving to: ‘train-data.tsv.3’

train-data.tsv.3    100%[===================>] 349.84K  --.-KB/s    in 0.01s   

2024-01-02 15:00:12 (28.3 MB/s) - ‘train-data.tsv.3’ saved [358233/358233]

--2024-01-02 15:00:12--  https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 172.67.70.149, 104.26.3.33, 104.26.2.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|172.67.70.149|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 118774 (116K) [text/tab-separated-values]
Saving to: ‘valid-data.tsv.3’

valid-data.tsv.3    100%[====

In [206]:
train_df = open(train_file_path, 'rb').read().decode(encoding='utf-8')
test_df = open(test_file_path, 'rb').read().decode(encoding='utf-8')
print(test_df[:100])
train_df = train_df.split('\n')
test_df = test_df.split('\n')
print(test_df[:100])
train_df = [x for x in train_df if x != '']
test_df = [x for x in test_df if x != '']
print(test_df[:100])


ham	i am in hospital da. . i will return home in evening
ham	not much, just some textin'. how bout y
['ham\ti am in hospital da. . i will return home in evening', "ham\tnot much, just some textin'. how bout you?", "ham\ti probably won't eat at all today. i think i'm gonna pop. how was your weekend? did u miss me?", 'ham\tdon‘t give a flying monkeys wot they think and i certainly don‘t mind. any friend of mine and all that!', 'ham\twho are you seeing?', 'ham\tyour opinion about me? 1. over 2. jada 3. kusruthi 4. lovable 5. silent 6. spl character 7. not matured 8. stylish 9. simple pls reply..', 'ham\tyesterday its with me only . now am going home.', "ham\tyes. it's all innocent fun. o:-)", 'ham\ta boy was late 2 home. his father: power of frndship', "ham\tis ur changes 2 da report big? cos i've already made changes 2 da previous report.", 'ham\tsorry about earlier. putting out fires.are you around to talk after 9? or do you actually have a life, lol!', "ham\ti'm still looking for a car

In [352]:
# Clean text and then one-hot-encode
import string
from nltk.stem.porter import PorterStemmer
from collections import Counter
from keras.preprocessing.text import one_hot
train_messages = []
test_messages = []
train_labels = []
test_labels = []

vocab_size = 30000

for line in train_df:
  # Break each line into words
  token = line.split('\t')[1].split()
  # Remove remaining tokens that are not alphabetic
  words = [w for w in token if w.isalpha()]
  # Stemming of words
  porter = PorterStemmer()
  stemmed = [porter.stem(w) for w in words]
  message = ' '.join(stemmed)
  # One-hot-encoding
  train_labels.append(line.split('\t')[0])
  train_messages.append(one_hot(message, vocab_size))

for line in test_df:
  # Break each line into words
  token = line.split('\t')[1].split()
  # Remove remaining tokens that are not alphabetic
  words = [w for w in token if w.isalpha()]
  # Stemming of words
  porter = PorterStemmer()
  stemmed = [porter.stem(w) for w in words]
  feature = ' '.join(stemmed)
  # One-hot-encoding
  test_labels.append(line.split('\t')[0])
  test_messages.append(one_hot(message, vocab_size))

print(Counter(train_labels))
print(Counter(test_labels))

Counter({'ham': 3619, 'spam': 560})
Counter({'ham': 1205, 'spam': 187})


In [353]:
train_labels = np.array([1 if label=='spam' else 0 for label in train_labels])
test_labels = np.array([1 if label=='spam' else 0 for label in test_labels])

In [354]:
print(train_labels)
print(test_labels)

[0 0 0 ... 1 1 0]
[0 0 0 ... 0 1 1]


In [355]:
print(train_messages)
print(test_messages)

[[21132, 5928, 3592, 15443, 29110, 4170, 26699, 728, 29915, 4170, 1662, 28020, 26699, 11949, 29767, 2149, 29110, 16336, 8066, 18332, 26699, 22449, 26730, 25652, 5336], [6200, 944, 772, 25547, 6857], [28020, 4170, 5932, 29915, 23436, 21134, 29405, 26699, 25077, 29313, 25882, 24181, 11544, 21846, 4170, 4959, 19223, 6577], [21274, 19620, 25080, 21417, 8259, 27322, 29972, 29972, 28968, 944, 16547, 3959, 25006, 24181, 15407], [772, 27533, 26699, 1534, 4694, 18188, 4224, 4234, 27052, 14935, 24181, 8960, 26782, 396], [2189, 11592, 12228, 29405, 2750, 16501, 19713, 14935, 7663, 12131, 11103, 5336, 6577, 18391, 26699, 944, 12608, 14343, 25882, 13528, 162, 4489, 2484, 21723, 47, 20789, 29214, 8889, 7830, 28968, 10196, 17576, 7860], [11399, 3880, 5397, 8389, 27052, 9810, 627, 8259, 13528], [12755, 20895, 28386, 8259, 16104, 7663], [7309, 363, 5397, 5397, 3, 19182, 772, 17861, 19488, 13422, 19413, 16773, 6827, 19854, 1109], [28020, 26117, 6200, 15543, 12131, 728, 1246, 24743, 5336, 6312, 17740, 11

In [356]:
from keras.preprocessing.sequence import pad_sequences

max_length = 200
train_padded = pad_sequences(train_messages, maxlen=max_length)
test_padded = pad_sequences(test_messages, maxlen=max_length)
print(train_padded[0])

[    0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0   

In [357]:
epochs = 10
# Create model
model = tf.keras.Sequential([
  tf.keras.layers.Embedding(vocab_size, 64),
  tf.keras.layers.LSTM(128),
  tf.keras.layers.Dense(256, activation="relu"),
  tf.keras.layers.Dense(128, activation="relu"),
  tf.keras.layers.Dense(1, activation="sigmoid")
])

model.summary()


Model: "sequential_36"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━┓
┃ Layer (type)                       ┃ Output Shape                  ┃     Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━┩
│ embedding_37 (Embedding)           │ ?                             │ 0 (unbuilt) │
├────────────────────────────────────┼───────────────────────────────┼─────────────┤
│ lstm_40 (LSTM)                     │ ?                             │ 0 (unbuilt) │
├────────────────────────────────────┼───────────────────────────────┼─────────────┤
│ dense_74 (Dense)                   │ ?                             │ 0 (unbuilt) │
├────────────────────────────────────┼───────────────────────────────┼─────────────┤
│ dense_75 (Dense)                   │ ?                             │ 0 (unbuilt) │
├────────────────────────────────────┼───────────────────────────────┼─────────────┤
│ dense_76 (Dense)                   │ ?                             │ 0 (unbuilt) │
└────────────────────────────────────┴───────────────────────────────┴─────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [358]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [359]:
history = model.fit(train_padded, train_labels, epochs=epochs, batch_size=128, validation_data=(test_padded, test_labels))

Epoch 1/10
33/33 ━━━━━━━━━━━━━━━━━━━━ 29s 768ms/step - accuracy: 0.8396 - loss: 0.4736 - val_accuracy: 0.8657 - val_loss: 0.3964
Epoch 2/10
33/33 ━━━━━━━━━━━━━━━━━━━━ 39s 714ms/step - accuracy: 0.9583 - loss: 0.1450 - val_accuracy: 0.8657 - val_loss: 0.4192
Epoch 3/10
33/33 ━━━━━━━━━━━━━━━━━━━━ 43s 763ms/step - accuracy: 0.9830 - loss: 0.0623 - val_accuracy: 0.8657 - val_loss: 0.5214
Epoch 4/10
33/33 ━━━━━━━━━━━━━━━━━━━━ 41s 755ms/step - accuracy: 0.9907 - loss: 0.0387 - val_accuracy: 0.8657 - val_loss: 0.6322
Epoch 5/10
33/33 ━━━━━━━━━━━━━━━━━━━━ 41s 760ms/step - accuracy: 0.9931 - loss: 0.0261 - val_accuracy: 0.8657 - val_loss: 0.6106
Epoch 6/10
33/33 ━━━━━━━━━━━━━━━━━━━━ 41s 761ms/step - accuracy: 0.9965 - loss: 0.0178 - val_accuracy: 0.8657 - val_loss: 0.6789
Epoch 7/10
33/33 ━━━━━━━━━━━━━━━━━━━━ 41s 748ms/step - accuracy: 0.9975 - loss: 0.0130 - val_accuracy: 0.8657 - val_loss: 0.7709
Epoch 8/10
33/33 ━━━━━━━━━━━━━━━━━━━━ 41s 727ms/step - accuracy: 0.9975 - loss: 0.0098 - val_accu

In [360]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):

  # Remove remaining tokens that are not alphabetic
  words = [w for w in pred_text.split() if w.isalpha()]
  # Stemming of words
  porter = PorterStemmer()
  stemmed = [porter.stem(w) for w in words]
  clean_text = ' '.join(stemmed)
  # Padding text and make prediction
  padded_text = pad_sequences([one_hot(clean_text, vocab_size)], maxlen=max_length)
  likeliness = model.predict(padded_text)[0][0]
  prediction = [likeliness, 'ham' if likeliness < 0.5 else 'spam']

  return (prediction)

pred_text = "how are you doing today?"

prediction = predict_message(pred_text)
print(prediction)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 340ms/step
[8.163298e-05, 'ham']


In [361]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    print(msg, ans)
    prediction = predict_message(msg)
    print(prediction)

    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()


how are you doing today ham
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
[9.897994e-06, 'ham']
sale today! to stop texts call 98912460324 spam
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
[0.9999999, 'spam']
i dont want to go. can we try it a different day? available sat ham
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
[4.511071e-05, 'ham']
our new mobile video service is live. just install on your phone to start watching. spam
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
[1.0, 'spam']
you have won £1000 cash! call to claim your prize. spam
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
[1.0, 'spam']
i'll bring it tomorrow. don't forget the milk. ham
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
[0.0032513815, 'ham']
wow, is your arm alright. that happened to me one time too ham
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
[1.7619589e-06, 'ham']
You passed the challenge. Great job!
